In [236]:
#interface graphique 
from tkinter import Tk, Canvas, Button
import pprint
import random
import time
#import de notebook
import nbimporter
from ia import Ia_dumb
Ia_dumb()

running


<h2><b>GAME</b></h2>

In [237]:
#function pour dessiner 
def pack_util(p) : 
    return p.pack()

In [238]:
#classe principale 
class Snake_game :
    def __init__(self, tinker, size = 500, play_btn = True):
        #initialisation des variables
        self.game_size = size
        self.pixel_size = 20
        self.tinker = tinker
        self.commands = {
            "z" : "top",
            "q" : "left",
            "d" : "right",
            "s" : "bottom"
        }
        self.reverse_commands = {
            "top" : "bottom",
            "right" : "left",
            "left" : "right",
            "bottom" : "top",
        }
        self.direction = ""
        self.commands_history = [self.direction]
        
        #initialisation fenêtre tkinter
        self.tinker.title("Snake")
        self.tinker.resizable(False, False)
        self.tinker.tk.call("tk", "scaling", 4.0)
        
        #initialisation de la matrice / creation d'une ligne vide
        self.grid_pos = [[] for i in range(0, int(self.game_size / self.pixel_size))]
        self.create_grid()
        
        #booleen pour afficher le bouton play 
        if play_btn : 
            #bouton jouer
            self.play_button = Button(self.tinker, text = "PLAY", command = self.play, width = 8, height = 2)
            pack_util(self.play_button)
            
    def create_canvas(self) :
        #canvas
        canvas = Canvas(self.tinker, width = self.game_size, height = self.game_size, background = "black", highlightthickness = 0)
        return canvas
          
    def create_grid(self) :
        
        #initialisation d'un dictionnaire
        pos_dict = {}             
        #remplir chaque ligne avec une liste de tuple de coordonnées possibles  
        for x in range(0, self.game_size + 1, self.pixel_size) :
            pos_dict[x] = []
            for y in range(0, self.game_size + 1, self.pixel_size) :
                pos_dict[x].append((x, y))

        #création de matrice      
        for x in list(pos_dict.keys())[:-1]:
            for i in range(len(pos_dict[x])-1):
                self.grid_pos[i].append([pos_dict[x][i],pos_dict[x+self.pixel_size][i+1]])
        
    #fonction d'écoute de touche    
    def set_key_event(self, event, canvas, apple) :
        #vérifier la touche 
        if event.char in self.commands :
            #définir la direction
            self.direction = self.commands[event.char]
            
            #on vérifie si l'utilisateur n'appuie pas deux fois sur la même touche
            if (self.commands_history[-1] != self.direction) and (self.commands_history[-1] != self.reverse_commands[self.direction]) : 
                self.commands_history.append(self.direction)
                self.snake.move(self.direction, canvas, self.commands_history, apple)
        
    def play(self) :
        print("Playing")
        #retirer le bouton play 
        self.play_button.destroy()
        
        #initialisation canvas
        self.canvas = self.create_canvas()
        
        #on vérifie si il y a plus de 1 élément fils sur la fenêtre
        if len(self.tinker.winfo_children()) == 1 :
            #initialisation de la pomme
            self.apple = Apple(self.tinker, self.game_size, False)
            self.apple.create_apple(self.canvas)
            
            #initialisation du snake
            self.snake = Snake(self.tinker, self.game_size, False)
            self.snake.create_snake(self.canvas)
            
            #écouter touche du clavier
            self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas, self.apple))
            
            pack_util(self.canvas)

In [239]:
#TODO set Apple 
class Apple (Snake_game):
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        self.init_pos = int((self.game_size/self.pixel_size)/2)

    def create_apple(self, canvas):
        print("apple", canvas)
        #création de la pomme
        self.apple = canvas.create_rectangle(
                self.grid_pos[self.init_pos][self.init_pos][0][1],
                self.grid_pos[self.init_pos][self.init_pos][0][0],
                self.grid_pos[self.init_pos][self.init_pos][1][1],
                self.grid_pos[self.init_pos][self.init_pos][1][0],
                outline="#000000",
                fill="#ff8f26",
                tags=('apple')
            )
    def update_apple(self, canvas, apple_id) :
        x = random.randint(5, int((self.game_size/self.pixel_size)/1.1))
        y = random.randint(5, int((self.game_size/self.pixel_size)/1.1))
        print(x,y,"appllllllle")
        #dessin du snake
        if x < len(self.grid_pos) :
            if y < len(self.grid_pos[x]):
                canvas.coords(
                    apple_id,
                    self.grid_pos[x][y][0][1],
                    self.grid_pos[x][y][0][0],
                    self.grid_pos[x][y][1][1],
                    self.grid_pos[x][y][1][0],
                )
                canvas.update()
        

In [250]:
class Snake (Snake_game) :
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        #initialisation position snake
        self.snake_head = (7, 5)
        self.snake_positions = [self.snake_head, (6,5), (5,5)]
        self.direction = "none"
        
    def create_snake(self, canvas):
        print("snake", canvas)
        for i, position in enumerate(self.snake_positions) :
            self.snake_part = canvas.create_rectangle(
                self.grid_pos[position[0]][position[1]][0][1],
                self.grid_pos[position[0]][position[1]][0][0],
                self.grid_pos[position[0]][position[1]][1][1],
                self.grid_pos[position[0]][position[1]][1][0],
                outline="#000000",
                fill="#1EC81C",
                tags=('snake_head') if i == 0 else ('snake')
            )
    def add_snake_part(self, canvas, x, y):
        if x < len(self.grid_pos) :
            if y < len(self.grid_pos[x]):
                self.snake_part = canvas.create_rectangle(
                        self.grid_pos[x][y][0][1],
                        self.grid_pos[x][y][0][0],
                        self.grid_pos[x][y][1][1],
                        self.grid_pos[x][y][1][0],
                        outline="#000000",
                        fill="#1EC81C",
                        tags=('snake')
                    )
    def update_snake(self, canvas, snake_id, x, y) :
        if x < len(self.grid_pos) :
            if y < len(self.grid_pos[x]):
                #dessin du snake
                canvas.coords(
                    snake_id,
                    self.grid_pos[x][y][0][1],
                    self.grid_pos[x][y][0][0],
                    self.grid_pos[x][y][1][1],
                    self.grid_pos[x][y][1][0],
                )
                canvas.update()
            
    def move(self, direction, canvas, commands_history, apple) :
        
        #controle de la boucle
        movement = True
        
        #changement de direction
        if len(commands_history) > 2 : 
            self.direction_is_diff = True if (direction == "right") or (direction == "left") else False
        else :
            self.direction_is_diff = False
        
        
        #boucle de mouvement
        while movement :
            #id tête du snake
            self.snake_head_ids = canvas.find_withtag("snake_head")

            #tout les ids du corps du snake
            self.snake_parts_ids = canvas.find_withtag("snake")

            #id de tout le snake
            self.snake_ids =  self.snake_parts_ids + self.snake_head_ids
            
            #id de la pomme
            self.apple_id = canvas.find_withtag("apple")
            
            for i, snake_id in enumerate(self.snake_ids):
                #coordonné d'une partie du snake par id
                coords = canvas.coords(snake_id)
                
                apple_coords = canvas.coords(self.apple_id)
                
                print("coords", coords)

                x = self.snake_positions[i][0]
                y = self.snake_positions[i][1]

                head_x = self.snake_head[0]
                head_y = self.snake_head[1]
                
                #mouvement à droite
                if direction == "right" :
                    y = head_y
                    #on vérifie que ça ne touche pas les bords 
                    if x < len(self.grid_pos):
                        if self.direction_is_diff:
                            x += i+1
                            if i == len(self.snake_ids) - 1 :
                                self.direction_is_diff = False
                                print("changed is diff", self.direction_is_diff)
                        else :
                            x += 1
                        self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head     
                        self.snake_positions[i] = (x,y)
                        self.update_snake(canvas, snake_id, x, y)
                    else :
                        movement = False
                        break
                            
                
                if direction == "left" :
                    y = head_y
                    if x < len(self.grid_pos):
                        if x > 0 :
                            print("x>0")
                            print("self.direction_is_diff", self.direction_is_diff)
                            if self.direction_is_diff :
                                x -= i+1
                                print("Index left",i)
                                if i == len( self.snake_ids) - 1 :
                                    self.direction_is_diff = False
                                    print("changed is diff", self.direction_is_diff)
                            else :
                                x -= 1
                            self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                            self.snake_positions[i] = (x,y)
                            self.update_snake(canvas, snake_id, x, y)
                        else :
                            movement = False
                            break
                    else :
                        movement = False
                        break
                        
                if direction == "bottom" :
                    x = head_x
                    if y < len(self.grid_pos[x]):
                        if self.direction_is_diff == False:
                            y += i+1
                            print("Index bottom",i)
                            if i == len (self.snake_ids) - 1 :
                                self.direction_is_diff = True
                                print("changed is diff", self.direction_is_diff)
                        else :
                            y += 1
                        self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                        self.snake_positions[i] = (x,y)
                        self.update_snake(canvas, snake_id, x, y)
                    else :
                        movement = False
                        break
                
                if direction == "top" :
                    x = head_x
                    if y < len(self.grid_pos[x]):
                        if y > 0:
                            if self.direction_is_diff == False:
                                y -= i+1
                                print("Index bottom",i)
                                if i == len (self.snake_ids) - 1 :
                                    self.direction_is_diff = True
                                    print("changed is diff", self.direction_is_diff)
                            else :
                                y -= 1
                            self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                            self.snake_positions[i] = (x,y)
                            self.update_snake(canvas, snake_id, x, y)
                        else :
                            movement = False
                            break
                    else :
                        movement = False
                        break
                #print("snake_head", self.snake_head_ids[0])
                #print(f"Snake_part-{snake_id} : {x, y}, direction : {direction}, self.direction_is_diff : {self.direction_is_diff}")
                
                if coords == apple_coords :
                    last_x = self.snake_positions[-1][0]
                    last_y = self.snake_positions[-1][1]
                    apple.update_apple(canvas, self.apple_id)
                    pos = (last_x, last_y + 1) if (direction == "right") or (direction == "left") else (last_x + 1, last_y)
                    self.snake_positions.append((pos[0], pos[1]))
                    self.add_snake_part(canvas, pos[0], pos[1])
                    
            time.sleep(0.2)
            #update du canvas
            #canvas.update()
            
        print(direction)

In [251]:
root = Tk()
snake_game = Snake_game(root)
root.mainloop()

Playing
apple .!canvas
snake .!canvas
coords [120.0, 100.0, 140.0, 120.0]
coords [100.0, 100.0, 120.0, 120.0]
coords [140.0, 100.0, 160.0, 120.0]
coords [160.0, 100.0, 180.0, 120.0]
coords [140.0, 100.0, 160.0, 120.0]
coords [120.0, 100.0, 140.0, 120.0]
coords [180.0, 100.0, 200.0, 120.0]
coords [160.0, 100.0, 180.0, 120.0]
coords [140.0, 100.0, 160.0, 120.0]
coords [200.0, 100.0, 220.0, 120.0]
coords [180.0, 100.0, 200.0, 120.0]
coords [160.0, 100.0, 180.0, 120.0]
coords [220.0, 100.0, 240.0, 120.0]
coords [200.0, 100.0, 220.0, 120.0]
coords [180.0, 100.0, 200.0, 120.0]
coords [240.0, 100.0, 260.0, 120.0]
coords [260.0, 100.0, 280.0, 120.0]
Index bottom 0
coords [220.0, 100.0, 240.0, 120.0]
Index bottom 1
coords [200.0, 100.0, 220.0, 120.0]
Index bottom 2
changed is diff True
coords [260.0, 120.0, 280.0, 140.0]
coords [260.0, 140.0, 280.0, 160.0]
coords [260.0, 160.0, 280.0, 180.0]
coords [260.0, 140.0, 280.0, 160.0]
coords [260.0, 160.0, 280.0, 180.0]
coords [260.0, 180.0, 280.0, 200

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-238-2452ec3fdef2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas, self.apple))
  File "<ipython-input-238-2452ec3fdef2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history, apple)
  File "<ipython-input-250-0964883bbda3>", line 150, in move
    if y < len(self.grid_pos[x]):
IndexError: list index out of range


right
coords [480.0, 280.0, 500.0, 300.0]
coords [480.0, 280.0, 500.0, 300.0]
x>0
self.direction_is_diff False
coords [480.0, 280.0, 500.0, 300.0]


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-238-2452ec3fdef2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas, self.apple))
  File "<ipython-input-238-2452ec3fdef2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history, apple)
  File "<ipython-input-250-0964883bbda3>", line 132, in move
    if y < len(self.grid_pos[x]):
IndexError: list index out of range


left
coords [460.0, 280.0, 480.0, 300.0]
coords [400.0, 280.0, 420.0, 300.0]
right


...

coords [460.0, 280.0, 480.0, 300.0]
coords [460.0, 280.0, 480.0, 300.0]
x>0
self.direction_is_diff False
coords [480.0, 280.0, 500.0, 300.0]


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-238-2452ec3fdef2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas, self.apple))
  File "<ipython-input-238-2452ec3fdef2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history, apple)
  File "<ipython-input-250-0964883bbda3>", line 150, in move
    if y < len(self.grid_pos[x]):
IndexError: list index out of range


left
coords [440.0, 280.0, 460.0, 300.0]
coords [440.0, 280.0, 460.0, 300.0]
coords [400.0, 280.0, 420.0, 300.0]


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-238-2452ec3fdef2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas, self.apple))
  File "<ipython-input-238-2452ec3fdef2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history, apple)
  File "<ipython-input-250-0964883bbda3>", line 132, in move
    if y < len(self.grid_pos[x]):
IndexError: list index out of range


right
